# Example of mod 3 Galois representations over an imaginary quadratic field

## This is an example where the $\mod 2$ representation is reducible and a $3$-adic method should work better than $2$-adic Faltings-Serre-Livne

### Define the number field:

In [1]:
x = polygen(QQ)
K.<i> = NumberField(x^2+1)

### Define the elliptic curve and related quantities.  This curve has LMFDB label [2.0.4.1-1312.1-b4](http://www.lmfdb.org/EllipticCurve/2.0.4.1/1312.1/b/4)

In [2]:
E = EllipticCurve([i + 1, i - 1, i + 1, -5*i, 2*i])
show(E)
S = E.conductor().prime_factors()
show(S)
S3 = S + K.primes_above(3)
show(S3)
f3 = E.division_polynomial(3)
show(f3)

Elliptic Curve defined by y^2 + (i+1)*x*y + (i+1)*y = x^3 + (i-1)*x^2 + (-5*i)*x + 2*i over Number Field in i with defining polynomial x^2 + 1

[Fractional ideal (i + 1), Fractional ideal (-5*i - 4)]

[Fractional ideal (i + 1), Fractional ideal (-5*i - 4), Fractional ideal (3)]

3*x^4 + (6*i - 4)*x^3 - 24*i*x^2 + 30*i*x - 10*i + 1

### Read in the files of code

In [3]:
%cd /home/jec/CremonaPacetti/code
%runfile S4.py
%runfile T0mod3.py
%runfile T0T1T2.py

/home/jec/CremonaPacetti/code


### Compute a set of test primes for the determinant character and use it to obtain the discriminant $\Delta$ such that $K(\sqrt{\Delta})$ is the fixed field of the kernel of the determinant character.  We know that this is $-3$ (modulo squares) for an elliptic curve.

In [4]:
T1, A, decoder = get_T1(K,S3)
print("T1:")
for P in T1:
    print("{}, norm {}".format(P,P.norm()))

BB = BlackBox_from_elliptic_curve(E)
decoder([0 if BB_det(BB)(P)%3==1 else 1 for P in T1])

T1:
Fractional ideal (-i - 2), norm 5
Fractional ideal (2*i + 1), norm 5
Fractional ideal (-3*i - 2), norm 13
Fractional ideal (2*i + 3), norm 13


3

### That's equivalent to $-3$ mod squares since $-1$ is a square.  We should get the same using the norms of the primes, since we know that for both the elliptic curve and the Bianchi newform (of weight $2$) the determinant of the image of the Frobenius at $P$ is $N(P)\pmod3$:

In [5]:
decoder([0 if P.norm()%3==1 else 1 for P in T1])

3

### Define the quadratic extension of K cut out by the determinant character: we know (and have just verified) that this is the $3$rd cyclotomic character:

In [6]:
xK = polygen(K)
M = K.extension(xK**2+3, 'r3')

### Find the irreducible quartics defining extensions of $K$ unramified outside $S3$ and containing $M$. Instead of finding a single list we find 3 lists, one for each Galois group, so that we'll know later which group we have:

In [7]:
S4quartics = S4_extensions_with_quadratic(K,S3,M)
D4quartics = D4_extensions_with_quadratic(K,S3,M)
C4quartics = C4_extensions_with_quadratic(K,S3,M)
quartics = S4quartics+D4quartics+C4quartics
print("There are {} candidate quartics, of which {} are S4, {} are D4 and {} are C4".format(len(quartics),len(S4quartics), len(D4quartics),len(C4quartics)))

There are 87 candidate quartics, of which 79 are S4, 0 are D4 and 8 are C4


### Compute $T_0$ and the associated $0$-$1$ vectors:

In [8]:
_, T0, vlist = get_T0_mod3(K,S3,quartics)
print("The test primes are")
for P in T0:
    print("P={} with norm {}".format(P,P.norm()))

The test primes are
P=Fractional ideal (4*i + 9) with norm 97
P=Fractional ideal (-8*i - 7) with norm 113
P=Fractional ideal (7*i + 8) with norm 113
P=Fractional ideal (4*i - 11) with norm 137
P=Fractional ideal (10*i - 7) with norm 149
P=Fractional ideal (-6*i - 11) with norm 157
P=Fractional ideal (6*i - 11) with norm 157
P=Fractional ideal (-2*i + 13) with norm 173
P=Fractional ideal (2*i + 13) with norm 173
P=Fractional ideal (-10*i - 9) with norm 181
P=Fractional ideal (9*i + 10) with norm 181
P=Fractional ideal (7*i + 12) with norm 193
P=Fractional ideal (i + 14) with norm 197


### Compute the $a_P$ for these primes from the elliptic curve:

In [9]:
aplist = [BB_trace(BB)(P) for P in T0]
v0 = [0 if ap%3==0 else 1 for ap in aplist]
print("The traces for these primes are {}".format(aplist))
print("Test vector = {}".format(v0))

The traces for these primes are [-2, -14, 2, 18, -2, -10, 14, -18, 6, -10, 18, -6, 6]
Test vector = [1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0]


### Test whether the test vector is in the list of test vectors.  If it is then the representation is irreducible and we can get the associated quartic from the table:

In [10]:
res = [i for i,vi in enumerate(vlist) if v0==vi]
if res:
    g3 = quartics[res[0]]
    print("The mod 3 representation is irreducible.")
    print("The splitting field of the projective representation is defined by the quartic {}".format(g3))
    group = 'S4' if g3 in S4quartics else 'D4' if g3 in D4quartics else 'C4'
    print("The projective image is isomorphic to {}".format(group))
else:
    print("The representation is reducible")

The mod 3 representation is irreducible.
The splitting field of the projective representation is defined by the quartic x^4 + (65184*i - 47046)*x^2 - 8*x - 1533324000*i - 508905987
The projective image is isomorphic to S4


### Check that we have recovered the 3-division field of $E$:

In [11]:
K.extension(f3,'t3').is_isomorphic(K.extension(g3,'u3'))

True

### Now we turn to the Bianchi modular form.  This requires running my C++ code (e.g. on Galois) to get the Hecke eigenvalues $a_P$ for $P$ dividing $\\{97,113,\dots,197\\}$:

In [12]:
%cd /home/jec/bianchi-progs
!echo "1 0 36 4 97 113 137 149 157 173 181 193 197 0 0 0" | /home/jec/bianchi-progs/moreap1

/home/jec/bianchi-progs
Program moreap1: for given field and level, assumes that the newforms file exists, and computes more individual Hecke eigenvalues.
---------------

Enter field: Verbose? Enter level: 
>>>> Level 1312.36.4 = (36+4i), norm = 1312 <<<<

2 newform(s) at level 1312.36.4 = (36+4i):
1:	basis = [];	aqlist = [ -1 -1 ];	aplist = [ 0 0 -2 -4 -6 2 -6 -4 10 0 -6 8 1 6 6 6 0 2 -10 -8 0 -8 -2 2 -16 2 -14 -2 8 -6 -14 -16 14 -2 -10 6 10 10 -14 18 -14 0 -16 22 2 18 -10 -20 2 0 14 10 18 -2 18 -18 -12 -30 18 12 0 -26 -10 -26 -6 -18 30 -10 16 22 22 -26 8 -14 -2 30 14 -18 -28 24 -12 -14 -2 -2 22 -26 -26 -6 -18 -18 -34 6 -4 2 -14 -42 22 2 -22 20 ]
Sign of F.E. = -1
Twisting prime lambda = 4+i, factor = 8
L/P ratio    = 0, cuspidal factor = 1
Integration matrix = [-9-18i,1;8-72i,3+2i], factor   = -1
2:	basis = [];	aqlist = [ 1 -1 ];	aplist = [ 0 2 -2 -2 6 -2 2 6 -2 10 6 2 1 -6 -6 -10 2 -2 14 -10 6 6 2 -14 -2 -18 -10 -2 10 -14 2 -10 -6 18 -2 -10 -10 14 -18 6 -10 18 -2 -6 6 -2 -10 10 -22

### From the above, taking the second eigenvalue of each pair (for form b, not form a) one can visually see that they agree with these:

In [13]:
zip(T0,aplist)

[(Fractional ideal (4*i + 9), -2),
 (Fractional ideal (-8*i - 7), -14),
 (Fractional ideal (7*i + 8), 2),
 (Fractional ideal (4*i - 11), 18),
 (Fractional ideal (10*i - 7), -2),
 (Fractional ideal (-6*i - 11), -10),
 (Fractional ideal (6*i - 11), 14),
 (Fractional ideal (-2*i + 13), -18),
 (Fractional ideal (2*i + 13), 6),
 (Fractional ideal (-10*i - 9), -10),
 (Fractional ideal (9*i + 10), 18),
 (Fractional ideal (7*i + 12), -6),
 (Fractional ideal (i + 14), 6)]

### That means, given that we know that this Bianchi newform has level equal to the conductor of $E$, namely $(36+4i)$, that the projective $\mod 3$ representation attached to the Bainchi newform also has full image with the same splitting field as for $E$.